<a href="https://colab.research.google.com/github/j-buss/wi-dpi-analysis/blob/development/eda/6.0_Salary_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Salary and Education in Wisconsin - 6.0 Salary Prediction


### Import Libraries

In [0]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
import seaborn as sns
import matplotlib.pyplot as plt

from google.cloud import bigquery

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [0]:
%matplotlib inline
plt.style.use('bmh')

### Functions

In [0]:
# To help with the pie charts later
def make_autopct(values):
  def my_autopct(pct):
    total = sum(values)
    val = int(round(pct*total/100.0))
    return '{p:.2f}% \n({v:,d})'.format(p=pct,v=val)
  return my_autopct

### Processing

In [0]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

In [0]:
project_id='wi-dpi-010'
landing_dataset_name='landing'
refined_dataset_name='refined'
gold_dataset_name='gold'

In [0]:
bq_client = bigquery.Client(project=project_id)

## Analysis


### Salary Linear Regression V1

In [0]:
select_2017 = '''
SELECT
  research_id,
  gender,
  race_ethnicity_cd,
  birth_year,
  contract_high_degree_cd,
  contract_days,
  contract_local_experience,
  contract_total_experience,
  cesa_num,
  assignment_fte,
  assignment_requires_dpi_license,
  assignment_staff_category_cd,
  position_classification,
  assignment_position_cd,
  assignment_area_cd,
  assignment_work_school_level_cd,
  salary
FROM
  {}.{}
WHERE
  assignment_fte = 1 and salary is not null
'''.format(refined_dataset_name, '2017_all_staff_report')

In [9]:
df = pd.read_gbq(select_2017, project_id=project_id)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value for dialect is changing to "standard" in a future version of pandas-gbq. Pass in dialect="legacy" to disable this warning.
  """Entry point for launching an IPython kernel.


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55633 entries, 0 to 55632
Data columns (total 17 columns):
research_id                        55633 non-null int64
gender                             55633 non-null object
race_ethnicity_cd                  55633 non-null object
birth_year                         55633 non-null int64
contract_high_degree_cd            55628 non-null object
contract_days                      55633 non-null int64
contract_local_experience          55633 non-null float64
contract_total_experience          55633 non-null float64
cesa_num                           55572 non-null object
assignment_fte                     55633 non-null float64
assignment_requires_dpi_license    55633 non-null object
assignment_staff_category_cd       55633 non-null object
position_classification            55633 non-null object
assignment_position_cd             55633 non-null object
assignment_area_cd                 55633 non-null object
assignment_work_school_level_cd    5

### One-hot encoding...

In [0]:
df_working = df

In [0]:
df_working = pd.concat([df_working.drop('gender', axis=1), pd.get_dummies(df_working['gender'],prefix='gender')], axis=1)
df_working = pd.concat([df_working.drop('race_ethnicity_cd', axis=1), pd.get_dummies(df_working['race_ethnicity_cd'],prefix='ethnicity')], axis=1)
df_working = pd.concat([df_working.drop('contract_high_degree_cd', axis=1), pd.get_dummies(df_working['contract_high_degree_cd'],prefix='degree')], axis=1)
df_working = pd.concat([df_working.drop('cesa_num', axis=1), pd.get_dummies(df_working['cesa_num'],prefix='cesa')], axis=1)
df_working = pd.concat([df_working.drop('assignment_staff_category_cd', axis=1), pd.get_dummies(df_working['assignment_staff_category_cd'],prefix='assignment_staff_category_cd')], axis=1)
df_working = pd.concat([df_working.drop('position_classification', axis=1), pd.get_dummies(df_working['position_classification'])], axis=1)
df_working = pd.concat([df_working.drop('assignment_work_school_level_cd', axis=1), pd.get_dummies(df_working['assignment_work_school_level_cd'],prefix='assignment_work_school_level_cd')], axis=1)

In [0]:
df_working = df_working.drop(['assignment_requires_dpi_license','assignment_position_cd','assignment_area_cd'], axis=1)

In [0]:
df_working.columns = df_working.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

#### Version 1

In [0]:
df_v1 = df_working[['research_id','gender','birth_year','contract_total_experience','salary']]
df_v1 = pd.concat([df_v1.drop('gender', axis=1), pd.get_dummies(df_v1['gender'],prefix='gender')], axis=1)

In [73]:
df_v1.head()

,research_id,birth_year,contract_total_experience,salary,gender_F,gender_M
0,39787,1981,2.0,102587.0,0,1
1,16230,1987,2.0,97668.0,0,1
2,138828,1969,2.0,40027.0,1,0
3,146690,1956,2.0,47390.0,1,0
4,16983,1989,2.0,43500.0,1,0


In [0]:
target_v1 = df_v1['salary']
data_v1 = df_v1.drop('salary', axis=1)

In [75]:
X_train, X_test, y_train, y_test = train_test_split(data_v1, target_v1, test_size=0.2, random_state=0)
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [76]:
model.score(X_test, y_test)

0.3752721160407724

#### Version 2

In [0]:
df_v2 = df_working[['research_id','gender','birth_year','contract_total_experience','salary']]
df_v2 = pd.concat([df_v2.drop('gender', axis=1), pd.get_dummies(df_v2['gender'],prefix='gender')], axis=1)

In [78]:
df_v2.head()

,research_id,birth_year,contract_total_experience,salary,gender_F,gender_M
0,39787,1981,2.0,102587.0,0,1
1,16230,1987,2.0,97668.0,0,1
2,138828,1969,2.0,40027.0,1,0
3,146690,1956,2.0,47390.0,1,0
4,16983,1989,2.0,43500.0,1,0


In [0]:
target_v2 = df_v2['salary']
data_v2 = df_v2.drop('salary', axis=1)

In [81]:
X_train, X_test, y_train, y_test = train_test_split(data_v2, target_v2, test_size=0.2, random_state=0)
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

### Test a new Point

In [0]:
test_point = [[2,1979,2.0,0,1]]

In [64]:
model.predict(test_point)

array([50982.9814335])

### Host a model

Evaluation: https://www.ritchieng.com/machine-learning-evaluate-linear-regression-model/

ML Engine - https://cloud.google.com/ml-engine/docs/scikit/using-pipelines